# Busca simples

Desenvolvimento de um buscador Simples: Booleano, TF-IDF, BM25

Tópicos abordados: Indexação, Bag-of-Words, TF-IDF, BM25

Aula 1 - [Unicamp - IA368DD: Deep Learning aplicado a sistemas de busca.](https://www.cpg.feec.unicamp.br/cpg/lista/caderno_horario_show.php?id=1779)

Autor: Marcus Vinícius Borela de Castro

[Repositório no github](https://github.com/marcusborela/deep_learning_em_buscas_unicamp)

[Link para chat de apoio com WebChatGPT](https://github.com/marcusborela/deep_learning_em_buscas_unicamp/blob/main/chat/CG%20uso%20no%20buscador%20aula%201.md)

[![Open In Colab latest github version](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/marcusborela/deep_learning_em_buscas_unicamp/blob/main/code/aula1_buscador_simples.ipynb) [Open In Colab latest github version]

## Enunciado exercício

Aula 2 - Notebook: Buscador Booleano/bag-of-words e buscador com TF-IDF

1. Usar o BM25 implementado pelo pyserini para buscar queries no TREC-DL 2020
Documentação referencia: https://github.com/castorini/pyserini/blob/master/docs/experiments-msmarco-passage.md
2. Implementar um buscador booleano/bag-of-words.
3. Implementar um buscador com TF-IDF
4. Avaliar implementações 1, 2, e 3 no TREC-DL 2020 e calcular o nDCG@10
Nos itens 2 e 3:

Fazer uma implementação que suporta buscar eficientemente milhões de documentos.

Não se pode usar bibliotecas como sklearn, que já implementam o BoW e TF-IDF.


## Organizando o ambiente

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Mar  4 15:05:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    52W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory


In [5]:
def mostra_memoria():
  vm = virtual_memory()
  ram={}
  ram['total']=round(vm.total / 1e9,2)
  ram['available']=round(virtual_memory().available / 1e9,2)
  # ram['percent']=round(virtual_memory().percent / 1e9,2)
  ram['used']=round(virtual_memory().used / 1e9,2)
  ram['free']=round(virtual_memory().free / 1e9,2)
  ram['active']=round(virtual_memory().active / 1e9,2)
  ram['inactive']=round(virtual_memory().inactive / 1e9,2)
  ram['buffers']=round(virtual_memory().buffers / 1e9,2)
  ram['cached']=round(virtual_memory().cached/1e9 ,2)
  print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")


In [6]:
mostra_memoria()

Your runtime RAM in gb: 
 total 89.64
 available 87.95
 used 0.9
 free 85.19
 cached 3.19
 buffers 0.36


### Vinculando pasta do google drive para salvar dados

In [10]:
import os

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!ls '/content/drive'

MyDrive


In [11]:
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /content


### Instalações de libraries

In [21]:
!pip install git+https://github.com/castorini/pygaggle.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/castorini/pygaggle.git to /tmp/pip-req-build-zx5uu6_g
  Running command git clone --filter=blob:none --quiet https://github.com/castorini/pygaggle.git /tmp/pip-req-build-zx5uu6_g
  Resolved https://github.com/castorini/pygaggle.git to commit c285f6084684367dd07b608ef19c2722b5b0637e
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 17.1 MB/s eta 0:00:00
    

In [22]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
!pip install faiss-cpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 81.8 MB/s eta 0:00:00


### Baixando o repositório do pyserini para usara seus scripts

In [15]:
path_pyserini = '/content/drive/MyDrive/treinamento/202301_IA368DD/code/pyserini'
path_pyserini_tools = path_pyserini + '/pyserini-master/anserini-tools-master'
path_pyserini_eval = path_pyserini + '/pyserini-master/pyserini/eval'

In [33]:
if not os.path.exists(path_pyserini):
    os.makedirs(path_pyserini)
    print('pasta criada')
    !wget -q https://github.com/castorini/pyserini/archive/refs/heads/master.zip -O pyserini.zip 
    !unzip -q pyserini.zip -d  {path_pyserini}
    # Baixando tools que é um atalho para https://github.com/castorini/anserini-tools
    !wget -q https://github.com/castorini/anserini-tools/archive/refs/heads/master.zip -O anserini-tools.zip 
    !unzip -q anserini-tools.zip -d  {path_pyserini}
path_pyserini = path_pyserini + '/pyserini-master'

In [16]:
 assert os.path.exists(path_pyserini), f"Pasta {path_pyserini} não criada!"

In [17]:
 assert os.path.exists(path_pyserini_tools), f"Pasta {path_pyserini_tools} não criada!"

In [18]:
 assert os.path.exists(path_pyserini_eval), f"Pasta {path_pyserini_eval} não criada!"

## Carga dos dados da TREC 2020 usando pyserini

### Obtendo dados a partir do pyserini


[Dicas aqui](https://github.com/castorini/pyserini/blob/master/docs/experiments-msmarco-passage.md)

In [19]:
path_data = '/content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage'

In [40]:
%%time
if not os.path.exists(path_data):
  os.makedirs(path_data)
  print('pasta criada')
  !wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz -P {path_data}
  !tar xvfz {path_data}/collectionandqueries.tar.gz -C {path_data}
  os.remove(f'{path_data}/collectionandqueries.tar.gz')
  print("Dados carregados!")
else:
  print("Dados já existiam!")    

In [24]:
 assert os.path.exists(path_data), f"Pasta {path_data} não criada!"

Passo anterior gera os seguintes arquivos:

* collection.tsv
* qrels.dev.small.tsv
* qrels.train.tsv
* queries.dev.small.tsv
* queries.dev.tsv
* queries.eval.small.tsv
* queries.eval.tsv
* queries.train.tsv

Next, we need to convert the MS MARCO tsv collection into Pyserini's jsonl files (which have one json object per line):

Baixando o programa do github do pyserini

In [ ]:
%%time
if not os.path.exists(f'{path_data}/collection_jsonl'):
  !python {path_pyserini_tools}/tools/scripts/msmarco/convert_collection_to_jsonl.py \
  --collection-path {path_data}/collection.tsv \
  --output-folder {path_data}/collection_jsonl
  print("Dados carregados!")
else:
  print("Dados já existiam!")    

Converting collection...
Converted 0 docs, writing into file 1
Converted 100,000 docs, writing into file 1
Converted 200,000 docs, writing into file 1
Converted 300,000 docs, writing into file 1
Converted 400,000 docs, writing into file 1
Converted 500,000 docs, writing into file 1
Converted 600,000 docs, writing into file 1
Converted 700,000 docs, writing into file 1
Converted 800,000 docs, writing into file 1
Converted 900,000 docs, writing into file 1
Converted 1,000,000 docs, writing into file 2
Converted 1,100,000 docs, writing into file 2
Converted 1,200,000 docs, writing into file 2
Converted 1,300,000 docs, writing into file 2
Converted 1,400,000 docs, writing into file 2
Converted 1,500,000 docs, writing into file 2
Converted 1,600,000 docs, writing into file 2
Converted 1,700,000 docs, writing into file 2
Converted 1,800,000 docs, writing into file 2
Converted 1,900,000 docs, writing into file 2
Converted 2,000,000 docs, writing into file 3
Converted 2,100,000 docs, writing i

In [27]:
assert os.path.exists(f'{path_data}/collection_jsonl'), f"Pasta {path_data}/collection_jsonl não criada!"

The above script should generate 9 jsonl files in collections/msmarco-passage/collection_jsonl, each with 1M lines (except for the last one, which should have 841,823 lines).

Convertendo as queries (small dev) para o formato trec para avaliações futuras

In [ ]:
if not os.path.exists(f'{path_data}/qrels.dev.small.trec'):
  !python {path_pyserini_tools}/scripts/msmarco/convert_msmarco_to_trec_qrels.py \
  --input {path_pyserini_tools}/topics-and-qrels/qrels.msmarco-passage.dev-subset.txt \
  --output {path_data}/qrels.dev.small.trec
  print("Conversão efetuada!")
else:
  print("Arquivo já existia!")


### Loading data in dicts

The 6980 queries in the development set are already stored in the repo. Let's take a peek:

In [ ]:
!head {path_pyserini_tools}/topics-and-qrels/topics.msmarco-passage.dev-subset.txt

1048585	what is paula deen's brother
2	 Androgen receptor define
524332	treating tension headaches without medication
1048642	what is paranoid sc
524447	treatment of varicose veins in legs
786674	what is prime rate in canada
1048876	who plays young dr mallard on ncis
1048917	what is operating system misconfiguration
786786	what is priority pass
524699	tricare service number


In [95]:
!head {path_pyserini_tools}/topics-and-qrels/topics.dl20.txt

1030303	who is aziz hashim
1037496	who is rep scalise?
1043135	who killed nicholas ii of russia
1045109	who owns barnhart crane
1049519	who said no one can make you feel inferior
1051399	who sings monk theme song
1056416	who was the highest career passer  rating in the nfl
1064670	why do hunters pattern their shotguns?
1065636	why do some places on my scalp feel sore
1071750	why is pete rose banned from hall of fame


In [ ]:
!head {path_data}/qrels.dev.small.trec

300674 0 7067032 1
125705 0 7067056 1
94798 0 7067181 1
9083 0 7067274 1
174249 0 7067348 1
320792 0 7067677 1
1090270 0 7067796 1
1101279 0 7067891 1
201376 0 7068066 1
54544 0 7068203 1


In [44]:
!head {path_pyserini_tools}/topics-and-qrels/qrels.msmarco-passage.dev-subset.txt

300674 0 7067032 1
125705 0 7067056 1
94798 0 7067181 1
9083 0 7067274 1
174249 0 7067348 1
320792 0 7067677 1
1090270 0 7067796 1
1101279 0 7067891 1
201376 0 7068066 1
54544 0 7068203 1


In [45]:
!head {path_pyserini_tools}/topics-and-qrels/qrels.dl20-passage.txt

23849 0 1020327 2
23849 0 1034183 3
23849 0 1120730 0
23849 0 1139571 1
23849 0 1143724 0
23849 0 1147202 0
23849 0 1150311 0
23849 0 1158886 2
23849 0 1175024 1
23849 0 1201385 0


Each line contains a tab-delimited (query id, query) pair. Conveniently, Pyserini already knows how to load and iterate through these pairs. We can now perform retrieval using these queries:

#### Carregando queries

In [61]:
def ler_arquivo_query_trec20(file_path:str):
  """
  Função para ler um arquivo de queries TREC 2020 e retorná-las em um dicionário.

  Args:
    file_path (str): Caminho do arquivo de queries TREC 2020

  Returns:
    dict: Dicionário em que as chaves são os IDs das queries e os valores são os
          textos das queries correspondentes.
  """

  # Cria um dicionário vazio para armazenar as queries
  query_dict = {}

  # Abre o arquivo em modo leitura
  with open(file_path, 'r') as f:
      
      # Itera sobre as linhas do arquivo
      for line in f:

          # Separa a linha em duas partes (id e texto), considerando que são separadas por uma tabulação
          query_id, query_text = line.strip().split('\t')
          query_id = int(query_id)
          # Adiciona a query ao dicionário, usando o id como chave e o texto como valor
          query_dict[query_id] = query_text

  # Retorna o dicionário com as queries
  return query_dict

Verificando queries de todo o dev dataset (total 6980)

In [99]:
query_dev_dict = ler_arquivo_query_trec20(f'{path_pyserini_tools}/topics-and-qrels/topics.msmarco-passage.dev-subset.txt')

In [102]:
len(query_dev_dict),list(query_dev_dict.items())[:4]

(6980,
 [(1048585, "what is paula deen's brother"),
  (2, ' Androgen receptor define'),
  (524332, 'treating tension headaches without medication'),
  (1048642, 'what is paranoid sc')])

Carregando o queries do trec20 dataset (total 200)

In [114]:
query_dev_dict = ler_arquivo_query_trec20(f'{path_pyserini_tools}/topics-and-qrels/topics.dl20.txt')

In [115]:
len(query_dev_dict),list(query_dev_dict.items())[:4]

(200,
 [(1030303, 'who is aziz hashim'),
  (1037496, 'who is rep scalise?'),
  (1043135, 'who killed nicholas ii of russia'),
  (1045109, 'who owns barnhart crane')])

Carregando usando get_topics:

In [144]:
from pyserini.search import get_topics

In [145]:
topics = get_topics('dl20')
print(f'{len(topics)} queries total')

200 queries total


In [149]:
len(topics), list(topics.items())[0]

(200, (735922, {'title': 'what is crimp oil'}))

#### Carregando qrel (relevância por query)

In [116]:
def ler_arquivo_qrels_trec20(file_path:str) -> dict:
    """
    Lê um arquivo TSV contendo a avaliação de relevância de documentos para cada consulta.
    
    Args:
    file_path: str - O caminho do arquivo a ser lido.
    
    Returns:
    dict - Um dicionário onde as chaves são os IDs das consultas e os valores são 
           dicionários em que as chaves são os IDs dos documentos e os valores são 
           os níveis de relevância (0, 1, 2, 3, ou 4) de cada documento para a consulta correspondente.
    """
    qrels_dict = {}

    with open(file_path, 'r') as f:
        # Itera sobre cada linha do arquivo
        for line in f:
            # Separa a linha em seus campos
            query_id, _, doc_id, relevance = line.strip().split()
            query_id = int(query_id)
            doc_id = int(doc_id)
            # Verifica se a consulta já existe no dicionário
            if query_id not in qrels_dict:
                qrels_dict[query_id] = {}
            # Adiciona o ID do documento e seu nível de relevância
            qrels_dict[query_id][doc_id] = int(relevance)
    return qrels_dict

Verificando qrel de todo o dev dataset

In [110]:
qrel_dev_dict = ler_arquivo_qrels_trec20(f'{path_data}/qrels.dev.small.trec')

In [111]:
len(qrel_dev_dict),list(qrel_dev_dict.items())[0]

(6980, (300674, {7067032: 1}))

Carregando o qrel do trec20 dataset

In [117]:
qrel_dev_dict = ler_arquivo_qrels_trec20(f'{path_pyserini_tools}/topics-and-qrels/qrels.dl20-passage.txt')

In [118]:
len(qrel_dev_dict),list(qrel_dev_dict.items())[0]

(54,
 (23849,
  {1020327: 2,
   1034183: 3,
   1120730: 0,
   1139571: 1,
   1143724: 0,
   1147202: 0,
   1150311: 0,
   1158886: 2,
   1175024: 1,
   1201385: 0,
   1215556: 0,
   1220759: 0,
   1221770: 0,
   1333480: 1,
   1381453: 2,
   1414114: 2,
   1414115: 0,
   1414120: 2,
   1449780: 0,
   146754: 0,
   1493231: 0,
   1532701: 0,
   1535484: 0,
   1605854: 1,
   1605857: 1,
   1622747: 1,
   17118: 0,
   17122: 0,
   1714915: 0,
   1714917: 1,
   1724687: 0,
   172488: 0,
   178252: 0,
   182049: 0,
   1827512: 1,
   1844627: 0,
   188190: 0,
   188246: 1,
   1944730: 0,
   2003292: 0,
   2017213: 0,
   2203364: 0,
   2209883: 0,
   2318793: 0,
   2339898: 1,
   2373852: 0,
   2397072: 0,
   2423771: 0,
   2516458: 0,
   2585563: 0,
   2593928: 0,
   2607127: 3,
   2607128: 1,
   2607129: 2,
   2607130: 2,
   2607131: 2,
   2607132: 3,
   2607134: 0,
   2647769: 3,
   2674124: 0,
   2766280: 0,
   282421: 0,
   2838462: 3,
   2880479: 0,
   2934343: 0,
   293608: 0,
   29375

Todas as 54 queries possuem informação de relevância

In [119]:
[query for query, doc_rel in list(qrel_dev_dict.items()) if len(doc_rel)==0]

[]

### Indexando Trec 2020 Collection using Pyserini

In [28]:
%%time
if not os.path.exists('./indexes/lucene-index-msmarco-passage'):
  !python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input {path_data}/collection_jsonl \
  --index indexes/lucene-index-msmarco-passage \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-03-04 15:30:08,085 INFO  [main] index.IndexCollection (IndexCollection.java:391) - Setting log level to INFO
2023-03-04 15:30:08,087 INFO  [main] index.IndexCollection (IndexCollection.java:394) - Starting indexer...
2023-03-04 15:30:08,087 INFO  [main] index.IndexCollection (IndexCollection.java:395) - ============ Loading Parameters ============
2023-03-04 15:30:08,088 INFO  [main] index.IndexCollection (IndexCollection.java:396) - DocumentCollection path: /content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage/collection_jsonl
2023-03-04 15:30:08,088 INFO  [main] index.IndexCollection (IndexCollection.java:397) - CollectionClass: JsonCollection
2023-03-04 15:30:08,088 INFO  [main] index.IndexCollection (IndexCollection.java:398) - Generator: DefaultLuceneDocumentGenerator
2023-03-04 15:30:08,089 INFO  [main] index.IndexCollection (IndexCollection.java:399) - Threads: 9
2023-03-04 15:30:08,089 INFO  [main] index.IndexCollection (IndexCollection.java:400) - 

In [46]:
!du -hs './indexes/lucene-index-msmarco-passage'

4.3G	./indexes/lucene-index-msmarco-passage


## Calculando ndcg@10 pelo pyserini no trec 2020 (small dev)

#### Com script

We can also use the official TREC evaluation tool, trec_eval, to compute metrics other than MRR@10. For that we first need to convert the run file into TREC format:



In [131]:
# trocar abaixo se for para realizar o search todo o dev dataset
# file_topics = 'msmarco-passage-dev-subset'
file_topics_search = 'dl20'
print(f'file_topics_search: {file_topics_search}')

# trocar abaixo se for para realizar o eval todo o dev dataset
#file_topics_eval = {path_data}/qrels.dev.small.trec
file_topics_eval = 'dl20-passage'
print(f'file_topics_eval: {file_topics_eval}')

file_topics_search: dl20
file_topics_eval: dl20-passage


In [121]:
num_max_hits = 100

In [135]:
%%time
!python -m pyserini.search.lucene \
  --index indexes/lucene-index-msmarco-passage \
  --topics {file_topics_search} \
  --output runs/run.msmarco-passage.bm25.trec \
  --output-format msmarco \
  --hits {num_max_hits} \
  --bm25 --k1 0.82 --b 0.68

Setting BM25 parameters: k1=0.82, b=0.68
Running dl20 topics, saving to runs/run.msmarco-passage.bm25.trec...
100% 200/200 [00:06<00:00, 28.65it/s]
CPU times: user 174 ms, sys: 36.7 ms, total: 211 ms
Wall time: 14 s


Here, we set the BM25 parameters to k1=0.82, b=0.68 (tuned by grid search). The option --output-format msmarco says to generate output in the MS MARCO output format. The option --hits specifies the number of documents to return per query. Thus, the output file should have approximately 6980 × num_max_hits (698.000, if it is 100) lines.

Retrieval speed will vary by hardware: On a reasonably modern CPU with an SSD, we might get around 13 qps (queries per second), and so the entire run should finish in under ten minutes (using a single thread). We can perform multi-threaded retrieval by using the --threads and --batch-size arguments. For example, setting --threads 16 --batch-size 64 on a CPU with sufficient cores, the entire run will finish in a couple of minutes.

Usamos parâmetro -l 2 seguindo orientação em pyserini\docs\experiments-msmarco-irst.md

(...)
Similarly, for TREC DL 2020:

```bash
python -m pyserini.eval.trec_eval -c -m map -m ndcg_cut.10 -l 2 \
  dl20-passage runs/run.irst-sum.passage.dl20.txt
```


In [134]:
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 -mrecall.100 -mmap -l 2 {file_topics_eval} runs/run.dl20-passage.bm25.trec

/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
msmarco run detected. Converting to trec...
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '-mrecall.100', '-mmap', '-l', '2', '/root/.cache/pyserini/topics-and-qrels/qrels.dl20-passage.txt', '/tmp/tmp_4ap4b29']
Results:
map                   	all	0.2695
recall_100            	all	0.5669
ndcg_cut_10           	all	0.4876


In [122]:
!python -m pyserini.search.lucene \
  --index indexes/lucene-index-msmarco-passage \
  --topics {file_topics} \
  --output runs/run.dl20-passage.bm25.trec \
  --output-format msmarco \
  --hits {num_max_hits} \
  --bm25 --k1 0.82 --b 0.68

Setting BM25 parameters: k1=0.82, b=0.68
Running dl20 topics, saving to runs/run.dl20-passage.bm25.trec...
100% 200/200 [00:07<00:00, 28.27it/s]


In [82]:
mostra_memoria()

Your runtime RAM in gb: 
 total 89.64
 available 87.14
 used 1.61
 free 54.17
 cached 33.39
 buffers 0.46


#### Com código

Adaptado do [caderno do colega Gustavo Bartz Guedes](https://colab.research.google.com/drive/10z86PObSxqbXczZ9pz0T-QurL21oMShf?usp=sharing)

In [154]:
# code from https://colab.research.google.com/github/castorini/anserini-notebooks/blob/master/pyserini_msmarco_passage_demo.ipynb
from pyserini.search import SimpleSearcher
from pyserini.search.lucene import LuceneSearcher
from tqdm import tqdm


In [156]:
# Run all queries in topics, retrive top 1k for each query
def run_all_queries(file, topics, searcher, num_max_hits=100):
    with open(file, 'w') as runfile:
        cnt = 0
        print('Running {} queries in total'.format(len(topics)))
        for id in tqdm(topics, desc='Running Queries'):
            query = topics[id]['title']
            hits = searcher.search(query, num_max_hits)
            for i in range(0, len(hits)):
                _ = runfile.write('{} Q0 {} {} {:.6f} Pyserini\n'.format(id, hits[i].docid, i+1, hits[i].score))
            cnt += 1
            if cnt % 100 == 0:
                print(f'{cnt} queries completed')


In [157]:
searcher = LuceneSearcher('./indexes/lucene-index-msmarco-passage')
searcher.set_bm25(k1=0.82, b=0.68)


In [158]:
run_all_queries('run-msmarco-passage-bm25.txt', topics, searcher, num_max_hits)

Running Queries:   2%|▏         | 4/200 [00:00<00:05, 38.06it/s]

Running 200 queries in total


Running Queries:  50%|█████     | 101/200 [00:03<00:02, 35.14it/s]

100 queries completed


Running Queries: 100%|██████████| 200/200 [00:06<00:00, 31.34it/s]

200 queries completed


In [159]:
!head run-msmarco-passage-bm25.txt

735922 Q0 7307871 1 10.128700 Pyserini
735922 Q0 7307863 2 10.118800 Pyserini
735922 Q0 8626892 3 10.041000 Pyserini
735922 Q0 8626890 4 9.854300 Pyserini
735922 Q0 2766952 5 9.461000 Pyserini
735922 Q0 8734268 6 9.248000 Pyserini
735922 Q0 8626887 7 8.954300 Pyserini
735922 Q0 7307868 8 8.540100 Pyserini
735922 Q0 180247 9 8.508000 Pyserini
735922 Q0 1428024 10 8.493600 Pyserini


##### Eval

In [160]:
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 -mrecall.100 -mmap -l 2 {file_topics_eval} run-msmarco-passage-bm25.txt

/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '-mrecall.100', '-mmap', '-l', '2', '/root/.cache/pyserini/topics-and-qrels/qrels.dl20-passage.txt', 'run-msmarco-passage-bm25.txt']
Results:
map                   	all	0.2695
recall_100            	all	0.5669
ndcg_cut_10           	all	0.4876


# OLD CODE

## Carga dos dados da TREC 2020 

Fonte: [Repositório do projeto Robustez query](https://github.com/leonardo3108/robustez-query)


Final Project at Discipline IA376, Deep Learning for NLP, Turma E - Tópicos em Engenharia de Computação VII - 2021.2S

Authors: Leonardo Augusto da Silva Pacheco e Marcus Vinícius Borela de Castro

[Mais sobre o dataset](https://microsoft.github.io/msmarco/)


### Extração dos julgamentos


Obtidos dados de teste do arquivo 2020qrels-pass.txt

Só para ciência, também há arquivos com relevância: 

. Dev: qrels.dev.tsv

. Train: qrels.train.tsv 

In [ ]:
! wget -nc https://trec.nist.gov/data/deep/2020qrels-pass.txt

--2023-03-03 23:16:29--  https://trec.nist.gov/data/deep/2020qrels-pass.txt
Resolving trec.nist.gov (trec.nist.gov)... 132.163.4.175, 2610:20:6b01:4::175
Connecting to trec.nist.gov (trec.nist.gov)|132.163.4.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 218617 (213K) [text/plain]
Saving to: ‘2020qrels-pass.txt’

2020qrels-pass.txt  100%[===================>] 213.49K  1.19MB/s    in 0.2s    

2023-03-03 23:16:29 (1.19 MB/s) - ‘2020qrels-pass.txt’ saved [218617/218617]



**Passages** were judged on a four-point scale of:
* Not Relevant (0), 
* Related (1), 
* Highly Relevant (2), and 
* Perfect (3), 

where 'Related' is actually NOT Relevant---it means that the passage was on the same general topic, but did not answer the question. 

Thus, for Passage Ranking task runs (only), to compute evaluation measures that use binary relevance judgments using 
trec_eval, you either need to use 
```
trec_eval's -l option [trec_eval -l 2 qrelsfile runfile]
```
or modify the qrels file to change all 1 judgments to 0.

In [ ]:
print('query_number passage_identifier judging')
queries = []
pids = []
for i, line in enumerate(open('2020qrels-pass.txt')):
    query_nr, _, pid, judging = line.rstrip().split()
    if query_nr not in queries:
        queries.append(query_nr)
    if pid not in pids:
        pids.append(pid)
    if i < 20: print(query_nr, pid, judging)

query_number passage_identifier judging
23849 1020327 2
23849 1034183 3
23849 1120730 0
23849 1139571 1
23849 1143724 0
23849 1147202 0
23849 1150311 0
23849 1158886 2
23849 1175024 1
23849 1201385 0
23849 1215556 0
23849 1220759 0
23849 1221770 0
23849 1333480 1
23849 1381453 2
23849 1414114 2
23849 1414115 0
23849 1414120 2
23849 1449780 0
23849 146754 0


In [ ]:
print('{} queries:'.format(len(queries)))
print(queries)

54 queries:
['23849', '42255', '47210', '67316', '118440', '121171', '135802', '141630', '156498', '169208', '174463', '258062', '324585', '330975', '332593', '336901', '390360', '405163', '555530', '583468', '640502', '673670', '701453', '730539', '768208', '877809', '911232', '914916', '938400', '940547', '997622', '1030303', '1037496', '1043135', '1051399', '1064670', '1071750', '1105792', '1106979', '1108651', '1109707', '1110678', '1113256', '1115210', '1116380', '1121353', '1122767', '1127540', '1131069', '1132532', '1133579', '1136043', '1136047', '1136962']


### Extração das Queries

Datasets extraídos a partir de: [TREC 2020 Deep Learning Track Guidelines](https://microsoft.github.io/msmarco/TREC-Deep-Learning-2020)

In [ ]:
! wget -nc https://msmarco.blob.core.windows.net/msmarcoranking/queries.tar.gz

--2023-03-03 23:23:09--  https://msmarco.blob.core.windows.net/msmarcoranking/queries.tar.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18882551 (18M) [application/gzip]
Saving to: ‘queries.tar.gz’

queries.tar.gz      100%[===================>]  18.01M  9.58MB/s    in 1.9s    

2023-03-03 23:23:11 (9.58 MB/s) - ‘queries.tar.gz’ saved [18882551/18882551]



In [ ]:
import gzip

query_text = {}

for i, line in enumerate(gzip.open('queries.tar.gz', mode='rt')):
    fields = line.strip().split()
    #if i not in [0, 101093, 202185]:
    #if i > 202185: print(i)
    number = fields[0]
    text = ' '.join(fields[1:])
    if number in queries:
        query_text[number] = text
for query in queries:
    print(query, query_text.get(query, '?????'))

23849 are naturalization records public information
42255 average salary for dental hygienist in nebraska
47210 average wedding dress alteration cost
67316 can fever cause miscarriage early pregnancy
118440 define bmt medical
121171 define etruscans
135802 definition of laudable
141630 describe how muscles and bones work together to produce movement
156498 do google docs auto save
169208 does mississippi have an income tax
174463 dog day afternoon meaning
258062 how long does it take to remove wisdom tooth
324585 how much money do motivational speakers make
330975 how much would it cost to install my own wind turbine
332593 how often to button quail lay eggs
336901 how old is vanessa redgrave
390360 ia suffix meaning
405163 is caffeine an narcotic
555530 what are best foods to lower cholesterol
583468 what carvedilol used for
640502 what does it mean if your tsh is low
673670 what is a alm
701453 what is a statutory deed
730539 what is chronometer who invented it
768208 what is mamey
8

In [ ]:
len(queries), len(query_text)

(54, 54)

### Extração das passagens do MsMarco

In [ ]:
! wget -nc https://msmarco.blob.core.windows.net/msmarcoranking/collection.tar.gz

--2023-03-03 23:25:32--  https://msmarco.blob.core.windows.net/msmarcoranking/collection.tar.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1035009698 (987M) [application/octet-stream]
Saving to: ‘collection.tar.gz’

collection.tar.gz   100%[===================>] 987.06M  6.73MB/s    in 1m 57s  

2023-03-03 23:27:30 (8.44 MB/s) - ‘collection.tar.gz’ saved [1035009698/1035009698]



In [ ]:
from tqdm import tqdm
import os

In [ ]:
%%time
passage_text = {}

for line in tqdm(gzip.open('collection.tar.gz', mode='rt'), unit='B', unit_scale=True, total=os.path.getsize('collection.tar.gz')):
    fields = line.strip().split()
    pid = fields[0]
    text = ' '.join(fields[1:])
    if pid in pids:
        passage_text[pid] = text
for ndx, pid in enumerate(pids):
    print(pid, passage_text.get(pid, '?????'))
    if ndx > 10: 
      break

  1%|          | 5.39M/1.04G [13:33<43:11:55, 6.62kB/s]


KeyboardInterrupt: ignored

In [ ]:
len(pids), len(passage_text)